# Gradient Boosting

![Tera](../imgs/tera.png)

# Revisitando a ideia de Bagging  e RF

In [1]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn
import pandas as pd
import numpy as np

%matplotlib inline
np.random.seed(42)

plt.rcParams['figure.figsize'] = (8.0, 5.0)

In [2]:
%load_ext autoreload
%autoreload 2

import utils

In [3]:
def sample(x, y, n):
    idx = np.random.randint(len(x), size=n)
    return x[idx], y[idx]

In [4]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

x, y = utils.load_dataset('regression')

FileNotFoundError: [Errno 2] No such file or directory: '../data/regression.p'

In [ ]:
plt.plot(x, y, '.', label="data")
N = 50
predictions = []
for r in range(N):
    
    sx, sy = sample(x, y, 20)
    
    dt = DecisionTreeRegressor(max_depth=5)
    dt.fit(sx, sy)
    
    ypred = dt.predict(x)
    predictions.append(ypred)
    
    #plt.step(x, ypred, label='run={}'.format(r))

ypred_bag = np.mean(np.array(predictions), axis=0)

plt.step(x, ypred_bag, label='bagging')
plt.legend()


![Comp](../imgs/random_forest.png)


## De volta ao Kaggle!

![Comp](../imgs/taxi-competition.png)


In [5]:
df = pd.read_csv('../data/kaggle/train.csv')

In [6]:
print('Instances x features:', df.shape)
df.head()

Instances x features: (1458644, 11)


,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [7]:
from sklearn.metrics import mean_absolute_error, mean_squared_log_error

def rmsle(ytrue, ypred):
    return np.sqrt(mean_squared_log_error(ytrue, ypred))

In [8]:
x = df.drop(['trip_duration', 'id', 'pickup_datetime', 'dropoff_datetime', 'store_and_fwd_flag'], axis=1)
y = df['trip_duration']

In [9]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.1, random_state=1)

In [12]:
%%time
from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor(max_depth=50, random_state=42)
reg.fit(xtrain, ytrain)

CPU times: user 2min 9s, sys: 1.41 s, total: 2min 11s
Wall time: 2min 10s


In [13]:
ypred = reg.predict(xtest)

In [23]:
print(mean_absolute_error(ytest, ypred)/60)
print(rmsle(ytest, ypred))

8.6241049227831
0.5800794344508592


# Mas dá pra fazer melhor?

In [33]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
%%time
reg = GradientBoostingRegressor(max_depth=50, random_state=42)
reg.fit(xtrain, ytrain)

In [15]:
ypred = reg.predict(xtest)

In [14]:
print(mean_absolute_error(ytest, ypred)/60)
print(rmsle(ytest, ypred))

8.591670309413301
0.5784431144809985


## Problema

![Comp](../imgs/process_gb_sklearn.png)

# Weak Learners vs Strong Learners

## Bagging (Random Forest)

- Bagging são ensembles que combinam Strong Learners. Eu calculo a média da combinação de várias, posso deixar elas crescerem uma vez que eu acredito que a média delas pode reduzir a **variância** do resultado.


![Comp](../imgs/bagging_example.png)

### Problema das RF

Ela tem limitações em relação ao problema de regressão: Não conseguem extrapolar além dos dados de treinamento

## Boosting

- Boosting, por sua vez, são ensembles que combina uma série de weak learners de forma a formar um modelo só. Mas como que boosting identifica essas "regras fracas"?

No caso de árvores, técnicas de boosting se baseiam no que é chamado de "Stump tree", ou seja, "árvores com apenas uma divisão". A cada iteração que o algoritmo é aplicado, cria-se uma nova divisão com base nos erros do passado (o modelo "presta mais atenção" nos erros que tivemos). Sendo assim, após várias iterações, essas "weak rules" são combinadas, formando um modelo final.

Sendo assim, pode-se dizer que Boosting tem uma atenção maior em modelos classificados incorretamente e, logo, ele gradativamente reduz o **bias** do resultado.


## FOTO BOOSTING EXAMPLE


## Bias-Variance Tradeoff


![Tera](../imgs/bias-variance.png)
http://scott.fortmann-roe.com/docs/BiasVariance.html

# Mas qual é melhor?

Depende !

## Alguns Desafios

(retirados do brilliant.org)

## A Temperatura 

Agora, você está trabalhando com a tentativa de prever qual a temperatura de amanhã. Você está tentando usar Árvores de Decisão Regressoras e, por algum motivo estranho, elas parecem que sempre superestimam o resultado. 

Nesse caso, qual dos dois é melhor? Bagging ou Boosting?

## O Peso

João está tentando advinhar o peso de algumas frutas, mas teve um problema. Seu primo é meio desligado e registrou alguns números, alguns muito baixos e alguns muito altos para algumas frutas aleatórias.

![](../imgs/bagging-boosting.png)


Ele quer usar um ensemble para fazer um modelo. Qual processo é melhor? Bagging ou Boosting?

# Indo um pouco mais a fundo

## A ideia do Boosting

Vou descrever um algoritmo que usa a ideia de Boosting de forma simples.
O algoritmo tem três parâmetros:
- $B$, o número de árvores
- $d$, o número de *splits*
- $\lambda$, o peso de cada árvore

Vamos ter um total de $n$ dados para treinamentos. Cada um desses dados é divido em duas partes, a parte preditora $x$ e a variável resposta, $y$. Disso, segue:

Inicialize uma função $f$ como $f(x) = 0$

Para cada $1 \leq b \leq B$:
 1. Crie um conjunto de resíduos $\{r_1, r_2, \dots, r_n\}$. Ou seja, um para cada dado, de forma que $r_i = y_i - f(x)$
 2. Encontre a árvore $f^b$ para os resíduos, parando após $d$ splits. Isso é feito em $f = f + \lambda * f^b$
 
 Em outras palavras, estamos aplicando um peso a cada árvore, durante cada interação (ou seja, de forma sequencial) de forma que a cada interação nós mudamos o nosso $f$ em função dos resíduos. 
 
 PS: É válido notar que no primeiro round a comparação é com a variável resposta.
 
 PS2: No fim dessas interações, nós "concatenamos" todas as árvores

## Exercício
(tirado do brilliant.org)

Abaixo, eu coloquei a primeira árvore gerada pelo processo de boosting descrito. 

![](../imgs/exs-boosting.png)

Dado $\lambda = 0.1$, qual será o resíduo do ponto azul quando ele for usado para treinar a árvore do round seguinte?